### Single epoch means one iteration with all the batchs in our training & testing set

### Trianing with multiple epochs

In [1]:
# # pip install import-ipynb
# import import_ipynb # i am importing this file from the other file.
# # from Training_WithSingle_Batch_PyTorch_CNN_calculate_loss_Gradient_Descent_update_weight import network

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)


train_set = torchvision.datasets.FashionMNIST(
    './FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()]) # converting ToTensor
)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)


len(train_set), len(train_loader)


(60000, 600)

In [6]:
sample = iter(next(train_loader))
image, label = sample
image

TypeError: 'DataLoader' object is not an iterator

In [3]:
class NetWork(nn.Module):
    
    def __init__(self):
        super(NetWork, self).__init__()
        
        #### conv layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)  
        
        #### Linear layers
        self.fc1 = nn.Linear(in_features= 12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120 , out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
            
        
    def forward(self, t):
        t = t #input layer
        
        #### hidden conv1 layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        #### hidden conv2 layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        #### hidden Linear_1 layer
        t = t.reshape(-1, 12*4*4)
        t = self.fc1(t)
        t = F.relu(t)
        
        #### hidden Linear_2 layer
        t = self.fc2(t)
        t = F.relu(t)
        
        #### hidden Linear_out layer
        t = self.out(t)
        
        return t
        

In [4]:
network = NetWork()

In [5]:
optimizer = optim.Adam(network.parameters(), lr=0.01)

In [6]:
#### will calu the num of correct prediction out of the batch_size
def get_num_corrected(pred, labels):
    return pred.argmax(dim=1).eq(labels).sum().item()


# For a single batch [ 100 inputs]

In [ ]:
# instead of the next(iter()) ------ we use for loop

total_loss = 0
total_corr = 0

for batch in train_loader: ### looping through 100 iter or batch w+ means 100 * 600 = 60,000
    #get batch
    images, labels = batch
    #pass batch
    preds = network(images)
    #calcu loss
    loss = F.cross_entropy(preds, labels)
    #calcu gradients
    optimizer.zero_grad()
    loss.backward()
    #update weights
    optimizer.step()
    
    total_loss += loss.item()
    total_corr += get_num_corrected(preds, labels)

    
print(f"total correct pred: {total_corr}")
print(f"total Error pred: {total_loss}")

In [ ]:
accur = 100 * total_corr / len(train_set)
print(f"accuracy: {accur} ")

# For multiple batchs ----- by useing epoch nums...

In [ ]:
# instead of the next(iter()) ------ we use for loop


for epoch in range(3):
    
    total_loss_2 = 0
    total_corr_2 = 0
    
    # single batch contains 600 iters
    for images, labels in train_loader: ### looping through 100 iter or batch w+ means 100 * batch_2(600) = 60,000
        #pass batch
        pred_scores = network(images)
        
        #calcu loss
        loss = F.cross_entropy(pred_scores, labels)
        
        #calcu gradients
        optimizer.zero_grad()
        loss.backward()
        
        #update weights
        optimizer.step()

        total_loss_2 += loss.item()
        total_corr_2 += get_num_corrected(pred_scores, labels)

    print(f"epoch:{epoch}, total correct pred:{total_corr_2}, total Error pred:{total_loss_2}")
    
accur_2 = 100 * total_corr_2 / len(train_set)
print(f"accuracy: {accur_2}")
